# Predicting ground state of molecule using Hybrid quantum-classical Neural Networks(FFNN) (please run all before look at it, I dont have time to train)
## Project Description - poig
I am a highschool student, unable find any free team so decide to solo it. This is my first AI project, without much knowledgth about AI.  
I dont have credit card, cant use aws power up redeem code.

my linkedin - linkedin.com/in/jun-liang-tan-0b8b53133/
### goal:
predict the ground state energies of molecules using Coulomb matrices as input using quantum-classical Neural Networks. I hope to combine advanced feed forward neural network(FFNN) and quantum neural network(QNN) to achieve higher accuracy.


It will train over 16,242 Molecules ground state with FNN&QNN.
[training databset from kaggle](https://www.kaggle.com/burakhmmtgl/energy-molecule/code)

### reference (those I think is important)- 
[1] [Predicting excited states from ground state wavefunction by supervised quantum machine learning Hiroki Kawai, Yuya O. Nakagawa](https://www.arxiv.org/pdf/2002.12925.pdf) (where my idea come from)   
[2] [Tree based machine learning framework for predicting ground state energies of molecules](https://www.arxiv.org/pdf/1609.07124.pdf)  (where the data set from, can download from kaggle)  
[3] [qiskit- hybrid neural network](https://www.qiskit.org/textbook/ch-machine-learning/machine-learning-qiskit-pytorch.html) (where I get idea how to do it)   
[4] [pennylane- recurrent neural network lstm](https://www.pennylane.ai/qml/demos/learning2learn.html#recurrent-neural-network-lstm) (where I get idea of####)  
[5] [github- AdvancedNNfromScratch](https://www.github.com/mukul-rathi/ChemRegressionNeuralNet/blob/master/AdvancedNNfromScratch.ipynb) (where I understand step need for this project)  
[6] [Can One Hear the Shape of a Molecule (from its Coulomb Matrix Eigenvalues)? Joshua Schrier Journal of Chemical Information and Modeling 2020 60 (8), 3804-3811 DOI: 10.1021/acs.jcim.0c00631](https://pubs.acs.org/doi/10.1021/acs.jcim.0c00631) (where I understand little about coulomb matrix, how much thing can do with it)  
[7] cheminformania.com/building-a-simple-qsar-model-using-a-feed-forward-neural-network-in-pytorch/

# Project
## Contents

1. [setup data](#setup-data)
2. [hyperparameter](#hyper-parameter-that-can-tune-to-improve-traning)
3.  
4. 

In [1]:
import qiskit
from qiskit import QuantumCircuit, transpile, Aer ,  execute, BasicAer, assemble
from qiskit.visualization import *

import torch
from torch.autograd import Function
import torchvision
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as u
from torch.utils.data import TensorDataset

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import networkx as nx
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy
#import sklearn.preprocessing import StandardScaler

from rdkit import Chem, DataStructs
from rdkit.Chem import PandasTools, AllChem

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

## setup data<a id="data"></a>
### Data set - kaggle.com/burakhmmtgl/energy-molecule

In [2]:
dataset = pd.read_csv('roboBohr.csv') #create a new dataframe containing the input data
dataset.Eat.describe()

count    16242.000000
mean       -11.178969
std          3.659133
min        -23.245373
25%        -13.475805
50%        -10.835211
75%         -8.623903
max         -0.789513
Name: Eat, dtype: float64

In [3]:
dataset.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,1267,1268,1269,1270,1271,1272,1273,1274,pubchem_id,Eat
0,0,73.516695,17.817765,12.469551,12.458130,12.454607,12.447345,12.433065,12.426926,12.387474,...,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,25004,-19.013763
1,1,73.516695,20.649126,18.527789,17.891535,17.887995,17.871731,17.852586,17.729842,15.864270,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25005,-10.161019
2,2,73.516695,17.830377,12.512263,12.404775,12.394493,12.391564,12.324461,12.238106,10.423249,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25006,-9.376619
3,3,73.516695,17.875810,17.871259,17.862402,17.850920,17.850440,12.558105,12.557645,12.517583,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25009,-13.776438
4,4,73.516695,17.883818,17.868256,17.864221,17.818540,12.508657,12.490519,12.450098,10.597068,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25011,-8.537140


### split- train set, validation, test

In [4]:
df = dataset.drop(['Unnamed: 0','pubchem_id'],axis=1)
X = df.drop(['Eat'], axis = 1).values
y = df['Eat'].values.reshape((-1,1))
print("X shape: ",X.shape)
print("Y shape: ",y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.10, random_state=42)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train,  test_size=0.05, random_state=42)
print("X train,test shape: ",X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print("X train,validation shape: ",X_train.shape, X_validation.shape, y_train.shape, y_validation.shape)

scaler = StandardScaler()
y_train = scaler.fit_transform(y_train)
y_test = scaler.transform(y_test)
y_validation = scaler.transform(y_validation)

X shape:  (16242, 1275)
Y shape:  (16242, 1)
X train,test shape:  (13886, 1275) (1625, 1275) (13886, 1) (1625, 1)
X train,validation shape:  (13886, 1275) (731, 1275) (13886, 1) (731, 1)


## create neural network
### hyper-parameter that can tune to improve traning

In [15]:
#can be easy tune when train
hyperparameters={}
hyperparameters["num_epochs"] = 20 #number of passes through the training set
hyperparameters["batch_size"] = 128 #number of examples trained upon in each step of training
#hyperparameters["layers_units"] = [X_train.T.shape[0], 128, 64, 32, 1] #layer 0 is the input layer
hyperparameters["hidden_size"] = 100
hyperparameters["input_size"] = X_train.T.shape[0]
hyperparameters["learning_rate"] = 1e-3 #learning rate
hyperparameters["dropout"] =0.8 #probability blacked out
hyperparameters["training_size"] = 0.01  #16,242 max

#hyperparameters["layers_units"]

In [6]:
print(X_train[:99,:].shape)
print(X_train.T.shape[0])

(99, 1275)
1275


### propagation

#### load account and dev

In [7]:
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
IBMQ.save_account('fb17afa861c2170c581e883349da83b1d508fa7a89d49714e76852f51f874de76988e70ad33b4a49b32d4fffb8f58c7c3abd2ab5be5d4b909bcc1af28e280848',overwrite=True)  #save your creds
qiskit.IBMQ.load_account()
provider = IBMQ.load_account()

C:\Users\junli\anaconda3\lib\site-packages\qiskit\providers\ibmq\ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '
ibmqfactory.load_account:WARNING:2022-02-27 01:24:53,477: Credentials are already in use. The existing account in the session will be replaced.


In [8]:
available_cloud_backends = provider.backends() 
print('\nHere is the list of cloud backends that are available to you:')
for i in available_cloud_backends: print(i)


Here is the list of cloud backends that are available to you:
ibmq_qasm_simulator
ibmq_armonk
ibmq_santiago
ibmq_bogota
ibmq_lima
ibmq_belem
ibmq_quito
simulator_statevector
simulator_mps
simulator_extended_stabilizer
simulator_stabilizer
ibmq_manila


#### Quantum circuit with qiskit (can apply complicated circuit, to archive quantum advantage)

In [9]:
class QuantumCircuit:
    def __init__(self, n_qubits, backend, shots):
        # --- Circuit definition ---
        self._circuit = qiskit.QuantumCircuit(n_qubits)
        
        all_qubits = [i for i in range(n_qubits)]
        self.theta = qiskit.circuit.Parameter('theta')
        
        self._circuit.h(all_qubits)
        self._circuit.barrier()
        self._circuit.ry(self.theta, all_qubits)
        
        self._circuit.measure_all()
        # ---------------------------

        self.backend = backend
        self.shots = shots
    
    def run(self, thetas):
        t_qc = transpile(self._circuit,
                         self.backend)
        qobj = assemble(t_qc,
                        shots=self.shots,
                        parameter_binds = [{self.theta: theta} for theta in thetas])
        job = self.backend.run(qobj)
        result = job.result().get_counts()
        
        counts = np.array(list(result.values()))
        states = np.array(list(result.keys())).astype(float)
        
        # Compute probabilities for each state
        probabilities = counts / self.shots
        # Get state expectation
        expectation = np.sum(states * probabilities)
        
        return torch.tensor(np.array([expectation]),dtype=torch.float64)

In [10]:
#test output
backend = qiskit.Aer.get_backend('qasm_simulator')
#backend = provider.get_backend('ibmq_qasm_simulator')

circuit = QuantumCircuit(1, backend, 100)
print('Expected value for rotation pi {}'.format(circuit.run([np.pi])[0]))
circuit._circuit.draw()

Expected value for rotation pi 0.52


┌───┐ ░ ┌───────────┐ ░ ┌─┐
   q_0: ┤ H ├─░─┤ RY(theta) ├─░─┤M├
        └───┘ ░ └───────────┘ ░ └╥┘
meas: 1/═════════════════════════╩═
                                 0

#### Quantum-Classical with PyTorch

In [11]:
#########
class HybridFunction(Function):
    @staticmethod
    def forward(ctx, input, quantum_circuit, shift):
        """ Forward pass computation """
        ctx.shift = shift
        ctx.quantum_circuit = quantum_circuit

        expectation_z = ctx.quantum_circuit.run(input[0].tolist())
        result = torch.tensor([expectation_z])
        ctx.save_for_backward(input, result)

        return result
        
    @staticmethod
    def backward(ctx, grad_output):
        """ Backward pass computation """
        input, expectation_z = ctx.saved_tensors
        input_list = np.array(input.tolist())
        
        
        shift_right = input_list + np.ones(input_list.shape) * ctx.shift
        shift_left = input_list - np.ones(input_list.shape) * ctx.shift
        
        gradients = []
        for i in range(len(input_list)):
            expectation_right = ctx.quantum_circuit.run(shift_right[i])
            expectation_left  = ctx.quantum_circuit.run(shift_left[i])
            
            gradient = torch.tensor([expectation_right]) - torch.tensor([expectation_left])
            gradients.append(gradient)
        gradients = np.array([gradients]).T
        return torch.tensor([gradients]).float() * grad_output.float(), None, None

class Hybrid(nn.Module):
    def __init__(self,qubit, backend, shots, shift):
        super(Hybrid, self).__init__()
        self.quantum_circuit = QuantumCircuit(qubit, backend, shots)
        self.shift = shift
        
    def forward(self, input):
        return HybridFunction.apply(input, self.quantum_circuit, self.shift)

#### Creating the Hybrid Neural Network


In [35]:
class HNN(nn.Module): 
    def __init__(self,inpt,hd,drop_out=hyperparameters["dropout"]): #'''layer_units=hyperparameters["layers_units"]'''
        super(HNN, self).__init__()
        #[setdata.drop(['Eat'], axis = 1).values.T.shape[0], 128, 64, 32, 1]
        #layer_1,layer_2,layer_3,layer_4,layer_5 = layer_units
        self.linear1 = nn.Linear(inpt, hd)
        self.linear2 = nn.Linear(hd, hd)
        self.linear3 = nn.Linear(hd, hd) 
        self.linear4 = nn.Linear(hd, 1) 
        self.hybrid = Hybrid(1,qiskit.Aer.get_backend('qasm_simulator'), 100, np.pi / 2)
        #normalization layer
        self.ln1 = nn.LayerNorm(hd)
        self.ln2 = nn.LayerNorm(hd)
        self.ln3 = nn.LayerNorm(hd)
        #self.hybrid2 = Hybrid(1,provider.get_backend('ibmq_qasm_simulator'), 100, np.pi / 2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(drop_out)
    def forward(self, x):
        out = self.linear1(x)
        out = self.ln1(out)
        out = self.relu(out)
        
        out = self.dropout(out)#drop_out
        
        out = self.linear1(x)
        out = self.ln1(out)
        out = self.relu(out)
        
        out = self.dropout(out)#drop_out
        
        out = self.linear3(out)
        out = self.ln3(out)
        out = self.relu(out)
        
        #out = self.dropout(out)
        out = self.linear4(out)
        #Final output layer
        #out = self.hybrid(out)
        #out = self.hybrid(out)
        return out

### evaluate model

In [36]:
def training_model(X_train,X_validation,y_train,y_validation, hyperparameters):
    num_epochs = hyperparameters["num_epochs"]#
    batch_size = hyperparameters["batch_size"]#
    #layers_units = hyperparameters["layers_units"] #bugs
    inpt = hyperparameters["input_size"]
    hd = hyperparameters["hidden_size"]
    learning_rate = hyperparameters["learning_rate"] #
    drop_out = hyperparameters["dropout"] # for dropout
    tra_size = hyperparameters["training_size"]
    tra_size=1
    trat_size =  int(np.round(tra_size* X_train.T.shape[0]))
    trav_size =  int(np.round(tra_size* X_validation.T.shape[0]))
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)

    
    X_train = torch.tensor(X_train[:trat_size,:], device=device).float()
    X_validation = torch.tensor(X_validation[:trav_size,:], device=device).float()
    
    y_train = torch.tensor(y_train[:trat_size,:], device=device).float()
    y_validation = torch.tensor(y_validation[:trav_size,:], device=device).float()
    
    train_dataset = TensorDataset(X_train, y_train)
    validation_dataset = TensorDataset(X_validation, y_validation)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                              batch_size=batch_size,
                                              shuffle=True)
    validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset,
                                              batch_size=batch_size,
                                          shuffle=False)
    
    criterion = nn.MSELoss()
    model = HNN(inpt, hd, drop_out)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    model.train() #Ensure the network is in "train" mode with dropouts active
    epochs = num_epochs
    for e in range(epochs):
        loss = 0
        for x, y in train_loader:
            # Training pass
            optimizer.zero_grad() # Initialize the gradients, which will be recorded during the forward pa
            
            output = model(x) #Forward pass of the mini-batch
            loss = criterion(output, y) #Computing the loss
            loss.backward() # calculate the backward pass
            optimizer.step() # Optimize the weights
             
            loss += loss.item()
        else:
            if e%10 == 0:
                #train_costs.append(loss)
                validation_loss = torch.mean(( y_validation - model(X_validation) )**2).item()
                #validation_loss.append(validation_loss)
                print("Epoch: %3i Training loss: %0.2F Validation loss: %0.2F"%(e,(loss/len(train_loader)), validation_loss))
    
    model.eval() #test
    y_pred_train = model(X_train)
    y_pred_validation = model(X_validation)
    y_pred_test = model(X_test)
    print("root_sqr_error")
    print("train error ",torch.mean(( y_train - y_pred_train )**2).item())
    print(torch.mean(( y_validation - y_pred_validation )**2).item())
    print(torch.mean(( y_test - y_pred_test )**2).item())
    def flatten(tensor):
        return tensor.cpu().detach().numpy().flatten()
    plt.scatter(flatten(y_pred_test), flatten(y_test), alpha=0.5, label="Test")
    plt.scatter(flatten(y_pred_train), flatten(y_train), alpha=0.1, label="Train")
    plt.legend()
    plt.plot([-1.5, 1.5], [-1.5,1.5], c="b")
    
    return model
    

### Training the Network

In [37]:
def training_model(X_train,X_validation,y_train,y_validation, hyperparameters):
    num_epochs = hyperparameters["num_epochs"]#
    batch_size = hyperparameters["batch_size"]#
    #layers_units = hyperparameters["layers_units"] #bugs
    inpt = hyperparameters["input_size"]
    hd = hyperparameters["hidden_size"]
    learning_rate = hyperparameters["learning_rate"] #
    drop_out = hyperparameters["dropout"] # for dropout
    tra_size = hyperparameters["training_size"]
    tra_size = 1
    trat_size =  int(np.round(tra_size* X_train.T.shape[0]))
    trav_size =  int(np.round(tra_size* X_validation.T.shape[0]))
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)
    
    X_train = torch.tensor(X_train[:trat_size,:], device=device).float()
    X_validation = torch.tensor(X_validation[:trav_size,:], device=device).float()
    
    y_train = torch.tensor(y_train[:trat_size,:], device=device).float()
    y_validation = torch.tensor(y_validation[:trav_size,:], device=device).float()
    
    train_dataset = TensorDataset(X_train, y_train)
    validation_dataset = TensorDataset(X_validation, y_validation)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                              batch_size=batch_size,
                                              shuffle=True)
    validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset,
                                              batch_size=batch_size,
                                          shuffle=False)
    
    criterion = nn.MSELoss()
    model = HNN(inpt, hd, drop_out)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    
    model.train() #Ensure the network is in "train" mode with dropouts active
    epochs = num_epochs
    for e in range(epochs):
        loss = 0
        for x, y in train_loader:
            # Training pass
            optimizer.zero_grad() # Initialize the gradients, which will be recorded during the forward pa
             
            output = model(x) #Forward pass of the mini-batch
            loss = criterion(output, y) #Computing the loss
            loss.backward() # calculate the backward pass
            optimizer.step() # Optimize the weights
             
            loss += loss.item()
        else:
            if e%10 == 0:
                validation_loss = torch.mean(( y_validation - model(X_validation) )**2).item()
                print("Epoch: %3i Training loss: %0.2F Validation loss: %0.2F"%(e,(loss/len(train_loader)), validation_loss))
    
    model.eval() #test
    y_pred_train = model(X_train)
    y_pred_validation = model(X_validation)
    y_pred_test = model(X_test)
    print("root_sqr_error")
    print("train error ",torch.mean(( y_train - y_pred_train )**2).item())
    print(torch.mean(( y_validation - y_pred_validation )**2).item())
    print(torch.mean(( y_test - y_pred_test )**2).item())

In [38]:
'''prev_parameters = copy.deepcopy(hyperparameters)
hyperparameters["training_size"] = 1
hyperparameters["num_epochs"] = 100
hyperparameters["learning_rate"] = 1e-2
hyperparameters["hidden_size"] = 100'''

training_model(X_train,X_validation,y_train,y_validation,hyperparameters)
#model.cuda()

cpu
Epoch:   0 Training loss: 0.10 Validation loss: 0.44
Epoch:  10 Training loss: 0.02 Validation loss: 0.12


AttributeError: 'numpy.ndarray' object has no attribute 'dim'

# Conculsion
## suppost to be look like
I want to run and compare hybrid in difference layer, but got a lot of bugs in the middle.
but I got a double bug when I want to impliment quantum layer to the circuit